In [1]:
%%capture
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

!pip install dask_cuda dask-cudf-cu12

In [2]:
import cudf
import cugraph as cg
import cupy as cp

import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Домашнее задание

В NetworkX есть алгоритм определения центральности через собственные вектора nx.algorithms.centrality.eigenvector_centrality(G). Сейчас мы рассмотрим реализацию данного алгоритма.
Заданием будет следующее:

    1) Переписать реализацию под GPU (cupy, numba, cudf, cugraph...)
    2) Для графа карате клуба вычислить degree centrality, betweenness centrality, eigenvector centrality и pagerank.
    3) Сравнить 4 характеристики между собой, на сколько они похожи \ отличаются и в чем
    
# Срок 02.12.2024
# Напоминаю, что курс про GPU и домашка также про GPU

### 1).

In [3]:
edges = pd.read_csv('karate-data.csv', sep='\t', names=['source', 'destination'])

In [4]:
G = nx.Graph()
G = nx.from_pandas_edgelist(edges, source='source', target='destination')

In [5]:
nx.algorithms.centrality.eigenvector_centrality(G)

{1: 0.35548349418519426,
 2: 0.26595387045450236,
 3: 0.3171893899684447,
 4: 0.21117407832057056,
 5: 0.0759664588165738,
 6: 0.07948057788594245,
 7: 0.07948057788594245,
 8: 0.1709551149803543,
 9: 0.22740509147166046,
 11: 0.0759664588165738,
 12: 0.05285416945233646,
 13: 0.08425192086558085,
 14: 0.22646969838808148,
 18: 0.0923967566684595,
 20: 0.14791134007618667,
 22: 0.0923967566684595,
 32: 0.19103626979791702,
 31: 0.17476027834493088,
 10: 0.10267519030637758,
 28: 0.13347932684333308,
 29: 0.13107925627221215,
 33: 0.3086510477336959,
 17: 0.02363479426059687,
 34: 0.37337121301323495,
 15: 0.10140627846270833,
 16: 0.10140627846270833,
 19: 0.10140627846270833,
 21: 0.10140627846270833,
 23: 0.10140627846270833,
 24: 0.15012328691726787,
 26: 0.05920820250279008,
 30: 0.13496528673866567,
 25: 0.057053735638028055,
 27: 0.07558192219009326}

In [6]:
M = nx.to_numpy_array(G)
eigenvalue, eigenvector = np.linalg.eig(M)
largest = np.array(eigenvector[:, 0].flatten().real)
norm = np.sign(largest.sum()) * np.linalg.norm(largest)
eigen = largest / norm


In [7]:
import numpy as np
def eigen_centrality(G):
    #переведем в матрицу numpy
    M = nx.to_numpy_array(G)
    #получим собственные вектора и собственные значения
    eigenvalue, eigenvector = np.linalg.eig(M)
    #берем вектор, соответсвующий наибольшему собственному значению
    largest = np.array(eigenvector[:, 0].flatten().real)
    #нормируем
    norm = np.sign(largest.sum()) * np.linalg.norm(largest)
    eigen = largest / norm
    return dict(zip(G, eigen))

In [8]:
eigen_centrality(G)

{1: 0.355491444524567,
 2: 0.26595991955249243,
 3: 0.31719250448643166,
 4: 0.21117972037789023,
 5: 0.075968818183069,
 6: 0.07948304511709949,
 7: 0.07948304511709949,
 8: 0.17095974804479627,
 9: 0.22740390712540007,
 11: 0.07596881818306894,
 12: 0.05285569749352141,
 13: 0.08425462871671373,
 14: 0.2264727201424812,
 18: 0.09239953819570265,
 20: 0.14791251029338756,
 22: 0.09239953819570265,
 32: 0.1910338414065437,
 31: 0.17475830231435285,
 10: 0.10267425072358635,
 28: 0.13347715338024002,
 29: 0.13107782298371043,
 33: 0.30864421979104734,
 17: 0.023635628104591376,
 34: 0.3733634702914828,
 15: 0.10140326218952443,
 16: 0.10140326218952443,
 19: 0.10140326218952443,
 21: 0.10140326218952443,
 23: 0.10140326218952443,
 24: 0.15011857186115274,
 26: 0.059206474916778565,
 30: 0.13496081926232797,
 25: 0.0570524405411657,
 27: 0.07557941348827213}

Gpu

In [9]:
edges_gpu = cudf.read_csv('karate-data.csv', sep='\t', names=['source', 'destination'])

In [10]:
#если не добавить веса, то будет все ломаться
edges_gpu['weights'] = 1

In [11]:
G_gpu = cg.Graph()
G_gpu.from_cudf_edgelist(edges_gpu, source='source', destination='destination', edge_attr='weights')

Функция, собирающая матрицу смежности. Делать это таким образом и с использованием edge_list приходится потому, что у cugraph есть только методы преобразования к numpy/pandas.

In [12]:
def make_adj_matrix(graph, edge_list):
    num_nodes = len(graph.nodes())
    adj_matrix = cp.zeros((num_nodes, num_nodes))
    for source, destination in edge_list[['source', 'destination']].to_cupy():
        adj_matrix[source - 1, destination - 1] = 1
    return adj_matrix

In [13]:
def eigen_centrality_gpu(G, edge_list):
    adj_matrix = make_adj_matrix(G, edge_list)
    eigenvalue, eigenvector = cp.linalg.eigh(adj_matrix)
    largest = cp.array(eigenvector[:, -1].flatten().real)
    norm = cp.sign(largest.sum()) * cp.linalg.norm(largest)
    eigen = largest / norm
    node_list = G.nodes().to_cupy().tolist()
    return dict(zip(node_list, eigen.tolist()))

In [14]:
eigen_centrality_gpu(G_gpu, edges_gpu)

{1: 0.35549144452456616,
 2: 0.2659599195524917,
 3: 0.31719250448643144,
 4: 0.21117972037789004,
 5: 0.07596881818306887,
 6: 0.0794830451170994,
 7: 0.07948304511709937,
 8: 0.17095974804479608,
 9: 0.22740390712540012,
 10: 0.10267425072358621,
 11: 0.07596881818306887,
 12: 0.052855697493521224,
 13: 0.08425462871671359,
 14: 0.22647272014248107,
 15: 0.10140326218952472,
 16: 0.10140326218952472,
 17: 0.023635628104591418,
 18: 0.09239953819570243,
 19: 0.10140326218952472,
 20: 0.14791251029338742,
 21: 0.10140326218952472,
 22: 0.09239953819570243,
 23: 0.10140326218952472,
 24: 0.15011857186115307,
 25: 0.057052440541165775,
 26: 0.05920647491677864,
 27: 0.07557941348827224,
 28: 0.13347715338024016,
 29: 0.1310778229837109,
 30: 0.13496081926232806,
 31: 0.174758302314353,
 32: 0.1910338414065438,
 33: 0.30864421979104756,
 34: 0.3733634702914836}

### 2, 3).

In [15]:
def get_centrality_by_degree(g):
    centrality = g.degree()
    centrality['degree_centrality'] = centrality['degree'] / (len(g.nodes())-1) / 2
    centrality = centrality.sort_values(by='degree_centrality', ascending=False)
    return centrality

In [16]:
centrality_by_degree = get_centrality_by_degree(G_gpu)
centrality_by_degree_list = centrality_by_degree['vertex'].to_arrow().to_pylist()

In [17]:
def get_centrality_by_betweenness(g):
    centrality = cg.centrality.betweenness_centrality(g)
    centrality = centrality.sort_values(by='betweenness_centrality', ascending=False)
    return centrality

In [18]:
centrality_by_betweenness = get_centrality_by_betweenness(G_gpu)

In [19]:
eigen_centrality = eigen_centrality_gpu(G_gpu, edges_gpu)
eigen_centrality_df = cudf.DataFrame(list(eigen_centrality.items()), columns=['vertex', 'eigen_centrality'])
eigen_centrality_df = eigen_centrality_df.sort_values(by='eigen_centrality', ascending=False)
eigen_centrality_df = eigen_centrality_df.reset_index(drop=True)

In [20]:
def get_pagerank(g):
    pagerank = cg.link_analysis.pagerank(G_gpu)
    pagerank = pagerank.sort_values(by='pagerank', ascending=False)
    return pagerank

In [21]:
pagerank = get_pagerank(G_gpu)

/usr/local/lib/python3.10/dist-packages/cugraph/link_analysis/pagerank.py:227: UserWarning: Pagerank expects the 'store_transposed' flag to be set to 'True' for optimal performance during the graph creation
  warnings.warn(warning_msg, UserWarning)


Таблица сравнения характеристик для каждой вершины

In [22]:
all_metrics = centrality_by_degree.set_index('vertex')\
    .join(centrality_by_betweenness.set_index('vertex'))\
    .join(eigen_centrality_df.set_index('vertex'))\
    .join(pagerank.set_index('vertex'))\
    .sort_values(by='degree_centrality', ascending=False)

In [23]:
all_metrics

,degree,degree_centrality,betweenness_centrality,eigen_centrality,pagerank
vertex,,,,,
34,34,0.515152,0.304075,0.373363,0.100917
1,32,0.484848,0.437635,0.355491,0.096999
33,24,0.363636,0.145247,0.308644,0.071692
3,20,0.303030,0.143657,0.317193,0.057078
2,18,0.272727,0.053937,0.265960,0.052877
4,12,0.181818,0.011909,0.211180,0.035860
32,12,0.181818,0.138276,0.191034,0.037158
9,10,0.151515,0.055927,0.227404,0.029766
14,10,0.151515,0.045863,0.226473,0.029537


Таблица сравнения списков вершин, осортированных по соответсвующим характеристикам

In [24]:
all_metrics_vertex_order = centrality_by_degree.drop(columns=['degree_centrality'])\
    .merge(centrality_by_betweenness.drop(columns=['betweenness_centrality']), left_index=True, right_index=True, suffixes=("_betweenness", ""))\
    .merge(pagerank.drop(columns=['pagerank']), left_index=True, right_index=True, suffixes=("_pagerank", ""))\
    .merge(eigen_centrality_df.drop(columns=['eigen_centrality']), left_index=True, right_index=True, suffixes=("_eigen", ""))\
    .sort_values(by='degree', ascending=False)

In [25]:
all_metrics_vertex_order

,degree,vertex_betweenness,vertex_pagerank,vertex_eigen,vertex
0,34,34,34,34,34
1,32,1,1,1,1
2,24,33,33,33,3
3,20,3,3,3,33
4,18,2,2,2,2
5,12,4,4,4,9
6,12,32,32,32,14
7,10,9,9,9,4
8,10,14,14,14,32
9,10,24,24,24,31


Как можно видеть из полученных таблиц, для всех 4 методов порядок вершин при сортировке по характеристике довольно похож (кроме centrality_by_degree). Сильнее всего отличается от трех других centrality_by_degree, это объясняется тем, что эта характеристика в отличии от других никак не учитывает глобальную структуру графа. Также у всех характеристик отличаются абсолютные значения и скорость падения. Можно также отметить, что единственная характеристика, где значение бывает 0 - это centrality_by_betweenness. Это связано с тем, что существуют вершины, через которые вообще не проходят пути.